In [424]:
from dotenv import load_dotenv
import os
import requests
import json
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import geopandas as gpd
from pymongo import MongoClient
import time
from cartoframes.viz import Map, Layer, popup_element

In [356]:
load_dotenv()

True

In [357]:
token = os.getenv("token")
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

In [4]:
#token, client_id, client_secret

In [408]:
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["Ironhack"]
companies = db.get_collection("companies")
companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies')

In [505]:
query_sf_1 = {"offices.0.city": "San Francisco", "funding_rounds.0.raised_amount": {"$gte": 1000000}}
projection_sf_1 = {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, "total_money_raised": 1, "offices": 1}

san_francisco = list(companies.find(query_sf_1, projection_sf_1))

for i in san_francisco:
    i.update({'latitude': i['offices'][0]['latitude'], 'longitude': i['offices'][0]['longitude']})
    del i['offices']

#san_francisco
df_sf_pre = pd.DataFrame(san_francisco)
df_sf=df_sf_pre.dropna(subset=['latitude', 'longitude'], thresh=1)


query_sf_2 = {"offices.0.city": "San Francisco", "category_code": "design"}
projection_sf_2 = {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, "total_money_raised": 1}

df_sf_design = pd.DataFrame(list(companies.find(query_sf_2, projection_sf_2)))

display(df_sf, df_sf_design)

df_sf.to_csv('df_1.csv', index=False)
df_sf_design.to_csv('df_2.csv', index=False)

,name,category_code,founded_year,total_money_raised,latitude,longitude
0,Digg,news,2004.0,$45M,37.764726,-122.394523
1,Twitter,social,2006.0,$1.16B,37.776805,-122.416924
2,StumbleUpon,web,2002.0,$18.5M,37.775196,-122.419204
3,Powerset,search,2006.0,$22.5M,37.778613,-122.395289
4,Technorati,advertising,2002.0,$32.1M,37.779558,-122.393041
...,...,...,...,...,...,...
286,UTOPY,software,1999.0,$7.5M,37.767575,-122.411157
287,Siluria Technologies,nanotech,2008.0,$103M,37.448491,-122.180281
288,ZoomSystems,hardware,2002.0,$87M,37.781689,-122.391061
290,TeachScape,software,1999.0,$16M,37.786905,-122.404392


,name,category_code,founded_year,total_money_raised
0,99designs,design,2008,$35M


In [504]:
query_bgl_1 = {"offices.0.city": "Bangalore", "funding_rounds.0.raised_amount": {"$gte": 1000000}}
projection_bgl_1 = {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, "total_money_raised": 1, "offices": 1}

bangalore = list(companies.find(query_bgl_1, projection_bgl_1))


for i in bangalore:
    i.update({'latitude': i['offices'][0]['latitude'], 'longitude': i['offices'][0]['longitude']})
    del i['offices']
    

df_bgl_pre = pd.DataFrame(bangalore)
#df_bgl
df_bgl=df_bgl_pre.dropna(subset=['latitude', 'longitude'], thresh=1)


query_bgl_2 = {"offices.0.city": "Bangalore", "category_code": "design"}
projection_bgl_2 = {"name": 1, "_id": 0}
df_bgl_design = pd.DataFrame(list(companies.find(query_bgl_2, projection_bgl_2)))


display(df_bgl, df_bgl_design)
df_bgl.to_csv('df_3.csv', index=False)

,name,category_code,founded_year,total_money_raised,latitude,longitude
0,Minglebox,web,2006,$7M,12.971606,77.594376
3,Guruji,search,2006,$15M,12.937677,77.565740
4,Ziva Software,mobile,2005,$1.7M,12.973784,77.643590
6,Manthan Systems,analytics,2003,$47M,12.970895,77.597675
7,Must See India,travel,2007,$5M,12.908470,77.600339
8,Althea Systems,hardware,2009,$3M,12.934960,77.613685


""


In [508]:
query_stk_1 = {"offices.0.city": "Stockholm", "funding_rounds.0.raised_amount": {"$gte": 1000000}}
projection_stk_1 = {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, 
                    "total_money_raised": 1, "offices": 1}

stockholm = list(companies.find(query_stk_1, projection_stk_1))


for i in stockholm:
    i.update({'latitude': i['offices'][0]['latitude'], 'longitude': i['offices'][0]['longitude']})
    del i['offices']
    

df_stk_pre = pd.DataFrame(stockholm)
#df_stk
df_stk=df_stk_pre.dropna(subset=['latitude', 'longitude'], thresh=1)


query_stk_2 = {"offices.0.city": "Stockholm", "category_code": "design"}
projection_stk_2 = {"name": 1, "_id": 0}
df_stk_design = pd.DataFrame(list(companies.find(query_stk_2, projection_stk_2)))

display(df_stk, df_stk_design)
df_stk.to_csv('df_4.csv', index=False)

,name,category_code,founded_year,total_money_raised,latitude,longitude
0,TestFreaks,ecommerce,2006,$5.8M,59.333991,18.057120
1,PacketFront,software,2001,€26M,59.332788,18.064488
2,Stardoll,games_video,2004,$10M,59.345553,18.037611
3,Bambuser,mobile,2007,$2.5M,59.332788,18.064488
6,Momail,mobile,2005,$5.4M,59.335981,18.075506
7,Syntensia,software,2005,$3.5M,59.612126,17.894767


""


In [520]:
df_count = pd.DataFrame({"San Francisco 1M$": [len(df_sf)], "San Francisco Design": [len(df_sf_design)], 
                         "Bangalore 1M$": [len(df_bgl)], "Bangalore Design": [len(df_bgl_design)],
                         "Stockhol 1M$": [len(df_sf)], "Stockholm Design": [len(df_stk_design)]})
df_count = df_count.set_index(["San Francisco 1M$", "San Francisco Design", 
                               "Bangalore 1M$", "Bangalore Design", 
                               "Stockhol 1M$", "Stockholm Design"])


df_count

,,,,,
San Francisco 1M$,San Francisco Design,Bangalore 1M$,Bangalore Design,Stockhol 1M$,Stockholm Design
239,1,6,0,239,0


In [147]:
sf_map = folium.Map(location=[df_sf_ll['latitude'][0], df_sf_ll['longitude'][0]], zoom_start=14)

for i, row in df_sf_ll.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(sf_map)

#sf_map

In [495]:
sf_lat = 37.7730148
sf_long = -122.4284791

sf_icon = folium.Icon(color="red", icon_color="yellow", icon="map-pin", prefix="fa")

map_sf = folium.Map(location=[sf_lat, sf_long], zoom_start=14)

folium.Marker(location=[sf_lat, sf_long], icon=sf_icon).add_to(map_sf)

map_sf.save("mp_sf.html")
map_sf

In [434]:
#StarBuck

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&chains=ab4c54c0-d68a-012e-5619-003048cad9da&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_star_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_star_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['chains'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_star_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_star_sf 

In [458]:
#Education 

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=12009&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_school_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_school_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_school_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_school_sf

In [445]:
#Airport

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=19031&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_airport_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_airport_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_airport_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_airport_sf 

In [448]:
#Party

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=13003&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_party_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_party_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_party_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_party_sf 

In [450]:
#Vegan Places

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=13377&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_vegan_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_vegan_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_vegan_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_vegan_sf 

In [456]:
#Basketball Court

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=18008&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_court_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_court_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_court_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_court_sf 

In [457]:
#Pets Hairdresser

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=15054&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_pet_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_pet_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_pet_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_pet_sf

In [523]:
df_sf_places = pd.concat([df_star_sf, df_school_sf, df_airport_sf, 
                          df_party_sf, df_vegan_sf, df_court_sf, df_pet_sf])

mean_lat_sf = df_sf_places['latitudes'].mean()
center_lat_sf = round(mean_lat_sf, 7)
mean_lon_sf = df_sf_places['longitudes'].mean()
center_lon_sf = round(mean_lon_sf, 7)

center_icon = folium.Icon(color="orange", icon_color="black", icon="star", prefix="fa")

#folium.Marker(location=[center_lat_sf, center_lon_sf], icon=center_icon).add_to(map_sf)
#map_sf


map_sf_mean = folium.Map(location=[center_lat_sf, center_lon_sf], zoom_start=14)


coordinates = list(zip(df_sf_places['latitudes'], df_sf_places['longitudes']))
HeatMap(coordinates).add_to(map_sf_mean)

map_sf_mean.save("map_sf_mean.html")
map_sf_mean

In [524]:
folium.Marker(location=[center_lat_sf, center_lon_sf], icon=center_icon).add_to(map_sf_mean)
map_sf_mean